In [1]:
import os
import sys
import xml.dom.minidom

# Make sure local modules can be imported
module_path_root = os.path.abspath(os.pardir)
if module_path_root not in sys.path:
    sys.path.append(module_path_root)

In [2]:
# Specify the locality whose OSM file we want to load

locality = 'Mount Eliza'

In [3]:
# Load the XML file into memory
# This assumes that we have reduced the OpenStreetMap data down to a small enough locality
# that the in-memory approach is feasible

osm_filename = os.path.join(os.pardir, 'data_sources', 'Locality_' + locality.replace(' ', '_') + '.osm')

doc = xml.dom.minidom.parse(osm_filename)

In [19]:
# Get lists of ways and nodes
ways = doc.getElementsByTagName('way')
print('Way count:  %d' % ways.length)
nodes = doc.getElementsByTagName('node')
print('Node count: %d' % nodes.length)



Way count:  1804
Node count: 16046


In [20]:
def find_way_by_name(name):
    for way in ways:
        tags = way.getElementsByTagName('tag')
        for tag in tags:
            k = tag.getAttribute('k')
            if k == 'name':
                v = tag.getAttribute('v')
                if v.upper() == name.upper():
                    return way

In [16]:
x = find_way('Millbank Drive')

In [21]:
def print_way(way):
    tags = way.getElementsByTagName('tag')
    for tag in tags:
        k = tag.getAttribute('k')
        if k == 'name':
            v = tag.getAttribute('v')
            print(v)
            return

In [22]:
print_way(x)

Millbank Drive


In [23]:
def find_ways_by_attribute(attr, exclusion_val_list=['no']):
    matching_ways = []
    
    for way in ways:
        tags = way.getElementsByTagName('tag')
        for tag in tags:
            k = tag.getAttribute('k')
            if k == attr:
                v = tag.getAttribute('v')
                for exclusion_val in exclusion_val_list:
                    if v == exclusion_val:
                        k = ''
            if k == attr:
                matching_ways.append(way)